<a href="https://colab.research.google.com/github/2653319/book-example/blob/main/%E5%B0%8D%E6%8A%97%E7%B6%B2%E8%B7%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#判別器   使用多層全連接網路建立  對抗網路
class discriminator(nn.Module):
  def __init__ (self):
    super(discriminator, self).__init__()
    self.dis = nn.Sequential(
        nn.Linear(784,256)
        nn.LeakyReLU(0.2),#斜率為0.2 的啟動函數
        nn.Linear(256,256),
        nn.LeakyReLU(0.2),
        nn.Linear(256,1),
        nn.Sigmoid()  #將結果對應到0~1之間機率進行二分法
    )
  def forward(self, x):
    x = self.dis(x)
    return x

In [ ]:
#生產器
class generator(nn.Module):  
  def __init__ (self,input_size):
    super(generator,self).__init__()
    self.gen = nn.Sequential(
        nn.Linear(input_size,256)
        nn.LeakyReLU(True),
        nn.Linear(256,256),
        nn.LeakyReLU(True),
        nn.Linear(256,784),
        nn.Tanh()  #使用函數Tanh() 將資料分布到-1~1之間 
    )
  def forward(self, x):
    x = self.gen(x)
    return x

In [ ]:
#定義損失函數和最佳化函數
criterion = nn.BCELoss()  #二分類的損失函數
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003) #使用Adam最佳化函式 學習率0.0003
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

In [ ]:
#判別器訓練
img = img.view(num_img, -1)
real_img = Variable(img).cuda()
real_label = Variable(torch.ones(num_img)).cuda() #真實資料  全1
fake_label = Variable(torch.zero(num_img)).cuda() #假資料  全0

real_out = D(real_img) #將真圖片丟進判別器 取得值real_out
d_loss_real = criterion(real_out, real_label)  #將real_out 和 real_label 丟入 計算並取得loss
#透過real_out和real_label的差距的loss 得知兩者差距為多少 用於改善
real_scores = real_out

z = Variable(torch.randn(num_img, z_dimension)).cuda()  #獲得隨機雜訊
fake_img = G(z)      #產生假圖片
fake_out = D(fake_img)   #經過判別器獲得的結果
d_loss_fake = criterion(fake_out, fake_label)  #將fake_out 和 fake_label 丟入 計算並取得 loss  
#透過fake_out和fake_label差距的loss 得知兩者差距為多少 用於改善

fake_scores = fake_out

d_loss = d_loss_real + d_loss_fake  #兩者loss相加 獲得的loss 用反向傳播 更新判斷器
d_optimizer.zero_grad()   #歸0梯度
d_loss.backward()     #反向傳播
d_optimizer.step()    #更新產生網路的參數

In [ ]:
#產生器訓練
z = Variable(torch.randn(num_img, z_dimension)).cuda()    #獲得隨機雜訊
fake_img = G(z)   #產生假圖片
output = D(fake_img)  #經過判別器獲得的結果
g_loss = criterion(output, real_label)  #獲得假圖片與真實圖片的label的loss  得知差距  用於改善

g_optimizer.zero_grad()   #歸0梯度
g_loss.backward()   #反向傳播
g_optimizer.step()   #更新產生網路的參數


In [ ]:
#使用旋積神經網路建立  對抗網路

In [ ]:
#判斷器
class discriminator(nn.Module):
  def __init__(self):
    super(discriminator, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, 5, padding = 2),  #batch, 32,28,28
        nn.LeakReLU(0.2,True),
        nn.AvgPool2d(2, stride=2)  #batch, 32,14,14
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, 5, padding = 2), #batch, 64,14,14
        nn.LeakReLU(0.2,True),
        nn.AvgPool2d(2, stride=2)   #batch, 64,7,7
    )
    self.fc = nn.Sequential(
        nn.Linear(64*7*7, 1024),  
        nn.LeakyReLU(0.2, True),
        nn.Linear(1024, 1),
        nn.Sigmoid()
    )
  def forward(self, x):
    '''
    x: batch, width, height, channel=1
    '''
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    return x

#產生器
class generator(nn.Module):
  def __init__ (self, input_size, num_feature):
    super(genertaor, self).__init__()
    self.fc = nn.Linear(input_size, num_feature) #batch, 3136 = 1 x 56 x 56
    self.br = nn.Sequential(
        nn.BatchNorm2d(1),
        nn.ReLU(True)
    )
    self.downsampe1 = nn.Sequential(
        nn.Conv2d(1, 50, 3, stride=1, padding=1),  #batch, 50,56,56
        nn.BatchNorm2d(50),
        nn.ReLU(True)
    )
    self.downsampe2 = nn.Sequential(
        nn.Conv2d(50, 25, 3, stride=1, padding=1), #batch, 25,56,56
        nn.BatchNorm2d(25),
        nn.ReLU(True)
    )
    self.downsampe3 = nn.Sequential(
        nn.Conv2d(25, 1, 2, stride=2),  #batch, 1,28,28
        nn.Tanh()
    )
  def forward(self, x):
    x = self.fc(x)
    x = x.view(x.size(0), 1, 56, 56)
    x = self.br(x)
    x = self.downsampe1(x)
    x = self.downsampe2(x)
    x = self.downsampe3(x)
    return x

In [ ]:
#判別器訓練
img = img.view(num_img, -1)
real_img = Variable(img).cuda()
real_label = Variable(torch.ones(num_img)).cuda() #真實資料  全1
fake_label = Variable(torch.zero(num_img)).cuda() #假資料  全0

real_out = D(real_img) #將真圖片丟進判別器 取得值real_out
d_loss_real = criterion(real_out, real_label)  #將real_out 和 real_label 丟入 計算並取得loss
#透過real_out和real_label的差距的loss 得知兩者差距為多少 用於改善
real_scores = real_out

z = Variable(torch.randn(num_img, z_dimension)).cuda()  #獲得隨機雜訊
fake_img = G(z)      #產生假圖片
fake_out = D(fake_img)   #經過判別器獲得的結果
d_loss_fake = criterion(fake_out, fake_label)  #將fake_out 和 fake_label 丟入 計算並取得 loss  
#透過fake_out和fake_label差距的loss 得知兩者差距為多少 用於改善

fake_scores = fake_out

d_loss = d_loss_real + d_loss_fake  #兩者loss相加
d_optimizer.zero_grad()   #歸0梯度
d_loss.backward()     #反向傳播
d_optimizer.step()    #更新產生網路的參數

In [ ]:
#產生器訓練
z = Variable(torch.randn(num_img, z_dimension)).cuda()    #獲得隨機雜訊
fake_img = G(z)   #產生假圖片
output = D(fake_img)  #經過判別器獲得的結果
g_loss = criterion(output, real_label)  #獲得假圖片與真實圖片的label的loss  得知差距  用於改善

g_optimizer.zero_grad()   #歸0梯度
g_loss.backward()   #反向傳播
g_optimizer.step()   #更新產生網路的參數